# Sanity Checker 3

#### utilizes the classes found in `PatchAnalyzer/utils/ephys_analyzer.py` and `PatchAnalyzer/model/data_loader.py` to calculate the following parameters:
* 1️⃣ Passive properties from VC (optional) & CC sweeps  
* 2️⃣ Spike detection → F–I curve & per-spike metrics  
* 3️⃣ One-click plot toggles for debugging  
* Copy-paste-ready cells; all user inputs live in the next “Config” cell


In [ ]:
# ---------------------------------------------------------------------
# 🛠️  USER CONFIG  – fill / tweak these and rerun the rest of the nb
# ---------------------------------------------------------------------


# 0. IMPORT NECCESSARY LIBRARYS

# 1. SET PATH FOR EXPERIMENT FOLDER

# 2. SELECT A SET OF PROTOCOLS USING CELL_ID

# 3. FLAGS TO DETERMINE VISUALIZATION AND VERBOSE RESULTS




In [ ]:
# ---------------------------------------------------------------------
# 🔄  LOAD METADATA + ALL TRACES for the chosen experiment
# ---------------------------------------------------------------------

# 0. LOAD VOLTAGE DATA FOR GIVEN CELL_ID

# 1. LOAD CURRENT DATA FOR GIVEN CELL_ID

# 2. BUILD APPROPRIATE OBJECTS TO FEED INTO VPROTANALYZER AND CPROTANALZYER

In [ ]:
# -----------------------------------------------------------------------------------------
# 🔍  CURRENT-CLAMP and VOLTAGE-CLAMP passive parameters – per sweep
# ----------------------------------------------------------------------------------------


# 1.A CALCULATE VOLTAGE PASSIVE PROPERTIES AND PRINT TO TABLE


# 2.A CALCULATE TEST PULSE REGION, PLOT (IF VERBOSE) AND PRINT TO TABLE

# 2.B  CALCULATE PASSIVE PROPERTIES FOR EACH SWEEP, PLOT TO BOX AND WISKERS WITH POINTS ONTOP (IF VERBOSE) AND PRINT TO TABLE

# req: plot the voltage values to the box and whiskers, but make it a different color point so it is easy to spot


In [ ]:
# ---------------------------------------------------------------------
# ⚡  SPIKE DETECTION  &  F–I CURVE  
# ---------------------------------------------------------------------

# 1. detect spikes, produce F-I curve with optional current/voltage clamp derived capacitance normalization (pA/pF) (if verbose)

#2. produce spike frequency and injected current table. table.

In [ ]:
# ---------------------------------------------------------------------
# 🔬  PER‑SPIKE METRICS  (half‑width, AHP, threshold, dV/dt‑max)
# ---------------------------------------------------------------------

# calculate all per spike metrics, plot mean values against injected current level on 3 seperate plots. if verbose
# produce table of spikes